In [2]:
import numpy as np
from Environment import GridworldEnv

Pretty printing has been turned OFF


In [3]:
env = GridworldEnv()

In [6]:
print("状态数：",env.nS)
print("动作数：",env.nA)
print("状态转移概率数组长度",len(env.P))
print(env.P[0]) # {action: [(prob, next_state, reward, isDone)]}

状态数： 16
动作数： 4
状态转移概率数组长度 16
{0: [(1.0, 0, -1.0, False)], 1: [(1.0, 1, -1.0, False)], 2: [(1.0, 4, -1.0, False)], 3: [(1.0, 0, -1.0, False)]}


In [58]:
init_policy = np.ones((env.nS, env.nA), dtype=np.float32) / env.nA
print("init policy:", init_policy)

init policy: [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]


In [69]:
def policy_eavl(policy, environment, discount = 1.0 , theta = 0.1):
    env = environment
    """
        input: policy (state_num, action_num), enviroment, discount, theta
        return: value_function (state_num)
    """
    
    V = np.zeros(env.nS, dtype=np.float32) # 价值函数初始化 (16)
    "1.遍历所有状态"
    max_epoch = 50
    for epoch in range(max_epoch):
        delta = 0. # delta用于每个epoch时进行比较
        for state in range(env.nS):
            v = 0
            "2.遍历该策略下可以采取的动作"
            for action, action_prob in enumerate(policy[state]):
                "3.遍历在该(策略、动作)可到达的下一个状态"
                for state_prob, next_state, reward, _ in env.P[state][action]:
                    v += action_prob * state_prob * (reward + discount * V[next_state])

            delta = np.maximum(delta, np.abs(V[state] - v))
            V[state] = v
            
        "打印调试信息"
        if(epoch % 10 == 0):
            print("epoch: %d, delta: %.2f." % (epoch, delta))
        if(delta <= theta):
            break
    print("Final epoch:", epoch)
    return V # (16)

In [70]:
V_pi = policy_eavl(init_policy, env)
print("V_pi:",V_pi.reshape(env.desc.shape))

epoch: 0, delta: 1.86.
epoch: 10, delta: 1.04.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
V_pi: [[-50.701645 -50.834114 -50.768112 -50.89453 ]
 [-50.51342  -50.       -49.50351  -50.      ]
 [-49.787567 -47.72631  -45.103867 -50.      ]
 [-50.       -43.69593  -30.746363   0.      ]]


In [75]:
def policy_improve(V_pi, environment, discount = 1.):
    env = environment
    """
        input: V_pi, enviroment
        return: improved_policy
    """
    improved_policy = np.zeros((env.nS, env.nA), dtype=np.float32)
    "1.对每个状态进行遍历"
    for state in range(env.nS):
        action_values = np.zeros(env.nA, dtype=np.float32)
        "2.对该状态下可能执行的动作进行遍历"
        for action in range(env.nA):    
            "3.对（状态，动作），遍历所有的下一个转态"
            for state_prob, next_state, reward, done in env.P[state][action]:
                if(done and next_state!=15): # 陷入到陷阱而非终点
                    action_values[action] = -np.inf
                    break
                else:
                    action_values[action] += state_prob * (reward + discount * V_pi[next_state])
        optim_action = np.argmax(action_values)
        # print("State: %d, action values: " % (state), action_values)
        improved_policy[state][optim_action] = 1. # 设置最优策略中optim_action索引为1，其他action为0
    
    return improved_policy

In [35]:
improved_policy = policy_improve(V_pi, env)
print(improved_policy)
print(np.argmax(improved_policy, axis=1).reshape(env.desc.shape))

[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
[[2 3 2 0]
 [2 0 2 0]
 [1 2 2 0]
 [0 1 1 0]]


In [72]:
init_policy = np.ones((env.nS, env.nA), dtype=np.float32) / env.nA
V_pi = policy_eavl(init_policy, env)
improved_policy = policy_improve(V_pi, env)
print(np.argmax(improved_policy, axis=1).reshape(env.desc.shape))
print(np.argmax(init_policy) == np.argmax(improved_policy))

epoch: 0, delta: 1.86.
epoch: 10, delta: 1.04.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
State: 0, action values:  [-51.701645 -51.834114 -51.51342  -51.701645]
State: 1, action values:  [-51.834114 -51.768112       -inf -51.701645]
State: 2, action values:  [-51.768112 -51.89453  -50.50351  -51.834114]
State: 3, action values:  [-51.89453  -51.89453        -inf -51.768112]
State: 4, action values:  [-51.701645       -inf -50.787567 -51.51342 ]
State: 5, action values:  [-inf -inf -inf -inf]
State: 6, action values:  [-51.768112       -inf -46.103867       -inf]
State: 7, action values:  [-inf -inf -inf -inf]
State: 8, action values:  [-51.51342  -48.72631        -inf -50.787567]
State: 9, action values:  [      -inf -46.103867 -44.69593  -50.787567]
State: 10, action values:  [-50.50351        -inf -31.746363 -48.72631 ]
State: 11, action values:  [-inf -inf -inf -inf]
State: 12, action values:  [-inf -inf -inf -inf]
State: 13, action valu

In [76]:
def policy_iteration(environment, init_policy, discount= 1.0):
    env = environment
    """
    策略迭代
        input: environment, init_policy, discount.
        return: optimized policy for current issue.
    """
    policy = init_policy.copy()
    max_episode = 10000
    for episode in range(max_episode):
        "策略评估"
        V_pi = policy_eavl(policy, env, discount)
        old_action = np.argmax(policy, axis=1)
        "策略提高"
        policy= policy_improve(V_pi, env, discount)
        optim_action = np.argmax(policy, axis=1)
        print("Episode%d: " % (episode))
        print(optim_action.reshape(env.desc.shape),"\n")

        "评判策略是否达到稳定"
        if (old_action == optim_action).all(): # 对np而言all表示与，any表示或
            break
    
    return policy

In [77]:
final_policy = policy_iteration(env, init_policy)
print("Final policy:", final_policy)

epoch: 0, delta: 1.86.
epoch: 10, delta: 1.04.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
Episode0: 
[[2 3 2 3]
 [2 0 2 0]
 [1 2 2 0]
 [0 1 1 0]] 

epoch: 0, delta: 2.00.
epoch: 10, delta: 1.00.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
Episode1: 
[[2 1 2 3]
 [2 0 2 0]
 [1 1 2 0]
 [0 1 1 0]] 

epoch: 0, delta: 2.00.
epoch: 10, delta: 1.00.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
Episode2: 
[[1 1 2 3]
 [2 0 2 0]
 [1 1 2 0]
 [0 1 1 0]] 

epoch: 0, delta: 2.00.
epoch: 10, delta: 1.00.
epoch: 20, delta: 1.00.
epoch: 30, delta: 1.00.
epoch: 40, delta: 1.00.
Final epoch: 49
Episode3: 
[[1 1 2 3]
 [2 0 2 0]
 [1 1 2 0]
 [0 1 1 0]] 

Final policy: [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0.